In [ ]:
# Comment: You might need to move this file to the root directory of the project to run it.
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame

from scipy.interpolate import interp1d

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from tsfresh.transformers import RelevantFeatureAugmenter, FeatureAugmenter
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters

from typing import Dict, Any
import numpy as np
import pandas as pd
from functools import partial

from utils_preprocessing import parse_circuit_params_from_str
from utils_preprocessing import preprocess_data
from utils_preprocessing import unwrap_df

Function definitions:

In [ ]:
def parse_circuit_params_from_str(params_str: str) -> Dict[str, float]:
    return {item.split(":")[0].strip(): float(item.split(":")[1].strip()) for item in params_str.split(",")}
    
def process_batch_element_params(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.values()))

def process_batch_element_params_str(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.keys()))

Process data:

In [ ]:
df = preprocess_data("./data/train_data.csv")
df_test = preprocess_data("./data/test_data.csv")

df["param_strs"] = df.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df["param_values"] = df.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)

df_test["param_strs"] = df_test.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df_test["param_values"] = df_test.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)


For tsfresh we need to 'unwrap' all the measurements into a dataframe with measurement ids, freq, zreal, and zimag columns

In [ ]:
df_ts = unwrap_df(df)
df_ts.shape, len(np.unique(df_ts["id"]))

df_ts_test = unwrap_df(df_test)
df_ts_test.shape, len(np.unique(df_ts_test["id"]))

Train a model for each class of circuit

In [ ]:
circuits = np.unique(df["Circuit"])
circuits

In [ ]:
dfs = list()
dfs_test = list()
for i in np.arange(len(circuits)):
  dfs.append(df[df["Circuit"] == circuits[i]])
  dfs_test.append(df_test[df_test["Circuit"] == circuits[i]])
dfs[0].head()

In [ ]:
circuit_str = []
for i in range(len(circuits)):
    for j in range(len(dfs[i]["param_strs"].iloc[0])):
        if j==0: 
            circuit_str.append(circuits[i])
        else: 
            circuit_str.append("")

In [ ]:
df_ = dfs[0]
df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])

In [ ]:
def ecm_regression(use_dummy=False, quantile=False, relevant=False):
    ppls = list()
    param_values_pred = list(range(df.shape[0]))
    predictions = []
    y_true = []
    circuit_str = []
    for i, [df_, df_test_] in enumerate(zip(dfs, dfs_test)):
        print(" ")
        print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
        df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
        df_ts_test_ = df_ts_test[df_ts_test["id"].isin(df_test[df_test["Circuit"] == df_["Circuit"].loc[df_.index[0]]].index)]
        X_train = pd.DataFrame(index=np.unique(df_ts_["id"]))
        y_train = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
        X_test = pd.DataFrame(index=np.unique(df_ts_test_["id"]))
        y_test = pd.DataFrame(df_test_['param_values'].to_list(), columns=df_test_["param_strs"].loc[df_test_.index[0]])
        # transformer mask, tell the transformer which parameters to skip over
        mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
        if not use_dummy:
            mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor(random_state=42))
        else: 
            mdl = MultiOutputRegressor(estimator=DummyRegressor(strategy='median'))

        if quantile:
            regr = TransformedTargetRegressor(regressor=mdl,
                                                transformer=QuantileTransformer(n_quantiles=10, random_state=0))
                                                #transformer=PowerTransformer())
        else:
            regr = mdl

        if relevant:
            ppl = Pipeline([
                    ('augmenter', RelevantFeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        else:
            ppl = Pipeline([
                    ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        ppl.set_params(augmenter__timeseries_container=df_ts_);
        ppl.fit(X_train, y_train)
        ppl.set_params(augmenter__timeseries_container=df_ts_test_);
        y_pred = ppl.predict(X_test)
        y_pred_df = pd.DataFrame(y_pred, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        y_true_df = pd.DataFrame(y_test, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        predictions.append(y_pred_df)
        y_true.append(y_true_df)
        print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
        p_strs = df_["param_strs"].loc[df_.index[0]]
        for j in np.arange(len(p_strs)):
            print("%s MAE:%.4g" % (p_strs[j], mean_absolute_error(y_test[p_strs[j]], y_pred[:,j])))
            if j==0: 
                circuit_str.append(circuits[i])
            else: 
                circuit_str.append("")
        ppls.append(ppl)
        #for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
            #param_values_pred[idx] = y_pred[i,:]
    return predictions, y_true, ppls, circuit_str

In [ ]:
predictions_xg, y_true, ppls_xg, circuit_str = ecm_regression(use_dummy=False, quantile=False)
predictions_xgt, y_true, ppls_xgt, circuit_str = ecm_regression(use_dummy=False, quantile=True)
predictions_d, y_true, ppls_d, circuit_str = ecm_regression(use_dummy=True, quantile=False)

In [ ]:
def MAE_loss(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def parse_mae(predictions, y_true, circuit_str, model_names, save=False, save_path=None):
    results = []
    k = 0
    for i in range(len(predictions[0])):
        for j, col in enumerate(predictions[0][i].columns):
            mae_test_element = []
            for prediction in predictions:
                mae_test_element.append(MAE_loss(y_true[i][col], prediction[i][col]))
            res_element = [circuit_str[k], col] + mae_test_element
            results.append(res_element)
            k += 1
    if save:
        # Save results to csv
        results_df = pd.DataFrame(results, columns=['Circuit', 'Parameter'] + model_names)
        results_df.to_csv(save_path+'results.csv', index=False)
    return results, results_df

In [ ]:
model_names = ['Dummy', 'tsfresh-XGBoost', 'tsfresh-XGBoost-Quantile']
results_xg, results_df = parse_mae([predictions_d, predictions_xg, predictions_xgt], y_true, circuit_str, model_names, save=True, save_path='reg-')

In [ ]:
results_df.to_latex('reg-results.tex', index=False)